In [ ]:
!pip install imutils
!pip install efficientnet

import pandas as pd
import numpy as np 
from tqdm import tqdm
import cv2
import os
import shutil
import itertools
import imutils
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix


import matplotlib as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from efficientnet.tfkeras import EfficientNetB0
from tensorflow.keras import layers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing import image
from keras.layers import Dense, Dropout, Flatten, BatchNormalization
import warnings
warnings.filterwarnings('ignore')

from pathlib import Path

img_dir = 'C:\\Users\\Mukesh\\Downloads\\archive (5)\\gaussian_filtered_images'
image_dir = Path(img_dir)
filepaths = list(image_dir.glob(r'**/*.png'))

labels = []
for filepath in filepaths:
    label = os.path.split(os.path.split(filepath)[0])[1]
    labels.append(label)

filepaths = pd.Series(filepaths, name='Filepath').astype(str)
labels = pd.Series(labels, name='Label')


df = pd.concat([filepaths, labels], axis=1)

df = df.sample(frac=1).reset_index(drop = True)

df.head(4)
df['Label'].value_counts()

import plotly.express as px

Class_series = pd.Series(df['Label'])
lis = ["No_DR", "Mild", "Moderate", "Severe", "Proliferate_DR"]

DR_or_not = Class_series.value_counts().tolist()

data = {'Severity': lis, 'Count': DR_or_not}
df_plot = pd.DataFrame(data)

fig = px.bar(df_plot, x='Count', y='Severity', orientation='h',
             color='Severity', color_discrete_sequence=["skyblue", "black", "pink", "purple", "blue"],
             title='Percentage among the different Severities of DR')
fig.update_layout(yaxis={'categoryorder': 'total ascending'})  
fig.show()


from sklearn.preprocessing import LabelEncoder
labelEncoder = LabelEncoder()
labelEncoder.fit(df['Label'])
df['Label']=labelEncoder.transform(df['Label'])

from sklearn.utils import resample

n_samples = 400
label_counts = df['Label'].value_counts()
dfs_by_label_resampled = {}

# Iterate through each label
for label in range(5):    
    # Check if there are enough samples for resampling
    if label_counts[label] < n_samples:
        # If there are not enough samples, duplicate the existing ones to reach 400
        df_label_resampled = pd.concat([df[df['Label'] == label]] * ((n_samples + label_counts[label] - 1) // label_counts[label]))
        df_label_resampled = df_label_resampled.sample(n_samples, replace=True, random_state=42)
    else:
        # If there are enough samples, resample to get exactly 400
        df_label_resampled = resample(df[df['Label'] == label], n_samples=n_samples, replace=True, random_state=42)
    
    # Append the resampled DataFrame to the dictionary
    dfs_by_label_resampled[label] = df_label_resampled

# Concatenate the resampled DataFrames
balanced_df = pd.concat(dfs_by_label_resampled.values())

# Shuffle the rows
balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)
balanced_df['Label'].value_counts()

import plotly.express as px

Class_series = pd.Series(balanced_df['Label'])
lis = ["No_DR", "Mild", "Moderate", "Severe", "Proliferate_DR"]

DR_or_not = Class_series.value_counts().tolist()

data = {'Severity': lis, 'Count': DR_or_not}
df_plot = pd.DataFrame(data)

fig = px.bar(df_plot, x='Count', y='Severity', orientation='h',
             color='Severity', color_discrete_sequence=["skyblue", "black", "pink", "purple", "blue"],
             title='Percentage among the different Severities of DR')
fig.update_layout(yaxis={'categoryorder': 'total ascending'})  
fig.show()

import glob

import glob
from PIL import Image

directory = '/kaggle/input/diabetic-retinopathy-224x224-gaussian-filtered/gaussian_filtered_images/gaussian_filtered_images/'

# Create a pattern to access all PNG files in the directory
pattern = os.path.join(directory, '*', '*.png')

# Use glob to get all file paths matching the pattern
image_paths = glob.glob(pattern)

size = (224, 224)


def load_image_and_resize(filepath):
        img = Image.open(filepath)
        img = img.resize(size)
        img = np.asarray(img)
        return img

balanced_df['image'] = balanced_df['Filepath'].map(lambda x: load_image_and_resize(x))

balanced_df['image']

import plotly.graph_objs as go
from plotly.subplots import make_subplots
sampled_images = balanced_df['image'].sample(6, random_state=42)
size = (224, 224)
sampled_images = [Image.fromarray(img).resize(size) for img in sampled_images]

sampled_images = [np.array(img) for img in sampled_images]

fig = make_subplots(rows=2, cols=3, subplot_titles=("Image 1", "Image 2", "Image 3", "Image 4", "Image 5", "Image 6"))

for i, img in enumerate(sampled_images, start=1):
    fig.add_trace(go.Image(z=img), row=(i-1) // 3 + 1, col=(i-1) % 3 + 1)

fig.update_layout(
    title="Sample Images",
    width=800,
    height=600,
    margin=dict(l=20, r=20, t=60, b=20),
)

fig.show()

from keras.utils import to_categorical
x=np.asarray(balanced_df['image'].to_list())
x=x/255
y=balanced_df['Label']
y=to_categorical(y,num_classes=5)
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.2,random_state=42,shuffle=True)


x

from tensorflow.keras.models import Model
from tensorflow.keras.applications.densenet import DenseNet121
from keras.callbacks import EarlyStopping


es = EarlyStopping(monitor='val_accuracy', min_delta = 0.005, patience=10, verbose=1, mode='auto')
vgg = DenseNet121(input_shape=(224,224,3),weights='imagenet',include_top=False)
vgg.trainable = False
x = Flatten()(vgg.output)
x = layers.Dense(256,activation='relu')(x)
x = layers.Dropout(0.4)(x)

prediction = Dense(5,activation='softmax')(x)
model = Model(inputs=vgg.input, outputs=prediction)
model.compile(Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train ,
                    y_train ,
                    epochs=70 ,
                    batch_size=8,
                    validation_data=(x_test , y_test) ,
                    callbacks=es
                    )

model_save_path = r"C:\Users\Mukesh\AppData\diabeticfolder\newmodel.h5"
model.save(model_save_path)
print(f'Model saved to {model_save_path}')

loss, accuracy = model.evaluate(x_test, y_test)

# Print the loss and accuracy
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

import matplotlib.pyplot as plt
tr_acc = history.history['accuracy']
tr_loss = history.history['loss']
val_acc = history.history['val_accuracy']
val_loss = history.history['val_loss']

epochs = [i+1 for i in range(len(tr_acc))]

plt.figure(figsize=(20, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs, tr_loss, 'r', label='Train Loss')
plt.plot(epochs, val_loss, 'g', label='Valid Loss')
plt.title('Loss')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Loss')

plt.subplot(1, 2, 2)
plt.plot(epochs, tr_acc, 'r', label='Train Accuracy')
plt.plot(epochs, val_acc, 'g', label='Valid Accuracy')
plt.title('Accuracy')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Accuracy')

plt.tight_layout()
plt.show()

from tensorflow.keras.models import load_model

# Path where your model is saved
model_path = r"C:\Users\Mukesh\AppData\diabeticfolder\newmodel.h5"

# Load the saved model
model = load_model(model_path)

from tensorflow.keras.preprocessing import image
import numpy as np

# Path to your single image file
img_path ='C:\\Users\\Mukesh\\Downloads\\archive (5)\\gaussian_filtered_images\\gaussian_filtered_images\\Proliferate_DR\\f2d2a0c92034.png'
# Load and preprocess the image
img = image.load_img(img_path, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)  # Expand dimensions to create batch of size 1
img_array /= 255.  # Normalize pixel values to [0, 1]

# Optionally, you may need to apply additional preprocessing like mean subtraction or scaling
# based on how the model was trained.

# Predict the class probabilities
predictions = model.predict(img_array)
predicted_class_index = np.argmax(predictions, axis=1)[0]

# Print the predicted class index (adjust as per your specific output)
print(f'Predicted class index: {predicted_class_index}')

# Optionally, if you have class labels, you can map the index to a label
class_labels = ['Mild', 'Moderate', 'No_DR', 'Proliferate_DR', 'Severe']  # Replace with your actual class labels
predicted_class_label = class_labels[predicted_class_index]

print(f'Predicted class label: {predicted_class_label}')


